In [34]:
import pandas as pd
import plotly.express as px
from plotly.io import show
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from fastf1.ergast import Ergast
import fastf1.plotting
import fastf1

In [361]:
def get_results(season):

    """wip"""

    ergast = Ergast()
    races = ergast.get_race_schedule(season)

    results = []

    # For each race in the season
    for rnd, race in races['raceName'].items():

        # Get results
        temp = ergast.get_race_results(season=season, round=rnd + 1).content[0]

        # If there is a sprint, get the results
        sprint = ergast.get_sprint_results(season=season, round=rnd + 1)

        if sprint.content and sprint.description['round'][0] == rnd + 1:

            # Merge sprint results
            temp = pd.merge(temp, sprint.content[0], on=['driverCode', 'constructorName'], how='left')

            # Add sprint points and race points to get the total
            temp['points'] = temp['points_x'] + temp['points_y']
            temp.drop(columns=['points_x', 'points_y'], inplace=True)

        # Add round number and Grand Prix name
        temp['round'] = rnd + 1
        temp['race'] = race.removesuffix(' Grand Prix')
        temp = temp[['round', 'race', 'constructorName', 'driverCode', 'points']]  # Keep useful cols.

        # Add to results
        results.append(temp)

    # Append all races into a single dataframe
    results = pd.concat(results)
    
    return results

In [362]:
df_2022 = get_results(2022)

In [286]:
def get_drivers_championship(df_results):

    """wip"""

    df = df_results.copy()

    df = df.pivot(index='driverCode', columns='round', values='points')
    # Here we have a 22-by-22 matrix (22 races and 22 drivers, incl. DEV and HUL)

    # Rank the drivers by their total points
    df['total_points'] = df.sum(axis=1)
    df = df.sort_values(by='total_points', ascending=False)
    df.drop(columns='total_points', inplace=True)

    # Use race name, instead of round no., as column names
    df.columns = df_results['race'].drop_duplicates()

    return df

Resultados de 2022. Pilotos

In [287]:
df_drivers_2022 = get_drivers_championship(df_2022)

Resultados equipos

In [289]:
def get_constructor_championship(df_results):

    """wip"""

    df = df_results.copy()

    # Agrupar por 'round', 'race' y 'constructorName', y sumar los puntos
    df = df.groupby(['round', 'race', 'constructorName'])['points'].sum().reset_index()

    df = df.pivot(index='constructorName', columns='round', values='points')

    # Rank the teams by their total points
    df['total_points'] = df.sum(axis=1)
    df = df.sort_values(by='total_points', ascending=False)
    df.drop(columns='total_points', inplace=True)

    # Use race name, instead of round no., as column names
    df.columns = df_results['race'].drop_duplicates()

    return df


In [290]:
df_constructors_2022 = get_constructor_championship(df_2022)

## Standings chart

In [298]:
def plot_standings_chart(df):

    """wip: cambiar los resaltes para equipos, paleta de colores y estilos"""

    fig = px.imshow(
        df,
        text_auto=True,
        aspect='auto',  # Automatically adjust the aspect ratio
        color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                                [0.25, 'rgb(107, 174, 214)'],
                                [0.5,  'rgb(33,  113, 181)'],
                                [0.75, 'rgb(8,   81,  156)'],
                                [1,    'rgb(8,   48,  107)']],
        labels={'x': 'Race',
                'y': 'Driver',
                'color': 'Points'}       # Change hover texts
    )

    # Remove axis titles
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')

    # Show all ticks, i.e. driver names
    fig.update_yaxes(tickmode='linear')

    # Show horizontal grid only
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                    showline=False,
                    tickson='boundaries')
    
    # Remove vertical grid
    fig.update_xaxes(showgrid=False, showline=False)

    # White background
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')     

    # Remove legend
    fig.update_layout(coloraxis_showscale=False)   

    # x-axis on top
    fig.update_layout(xaxis=dict(side='top'))           

    # Remove border margins
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))  

    show(fig)

In [294]:
plot_standings_chart(df_drivers_2022)

Hay que cambiar los resaltes para equipos

In [296]:
plot_standings_chart(df_constructors_2022)

---

In [22]:
def championship(results):

    drivers = results.index.to_list()

    fig, ax = plt.subplots(figsize=(8, 5))


    for drv in drivers:
        drv_laps = session.laps.pick_drivers([drv])

        try:
            style = fastf1.plotting.get_driver_style(identifier=abb,
                                                    style=['color', 'linestyle'],
                                                    session=session)

            ax.plot(drv_laps['LapNumber'], drv_laps['Position'],
                    label=abb, **style)
            
        except IndexError:
            print('Driver not found. Probably DNS')
        
    ax.set_ylim([20.5, 0.5])
    ax.set_yticks([1, 5, 10, 15, 20])
    ax.set_xlabel('Lap')
    ax.set_ylabel('Position')
    ax.set_title(f'Position changes | {title}')

    ax.legend(bbox_to_anchor=(1.0, 1.02))
    plt.tight_layout()

In [305]:
df_cumulative_drivers_2022 = df_drivers_2022.fillna(0).cumsum(axis=1)

In [24]:
session = fastf1.get_session(2022, 'Monza', 'R')  # Ejemplo: Gran Premio de Italia, Carrera
session.load()

core           INFO 	Loading data for Italian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '44', '11', '4', '10', '45', '24', '31', '47', '77', '22', '6', '20', '3', '18', '14', '5']


Plotly

In [321]:
def plot_drivers_championship(df_cumulative, session, top=None):

    """wip"""

    gps = df_cumulative.columns
    if top:
        drivers = df_cumulative.index[:top]
    else:
        drivers = df_cumulative.index

    fig = go.Figure()

    for driver in drivers:
        points = df_cumulative.loc[driver, gps]
        
        try:
            style = fastf1.plotting.get_driver_style(driver, style=['color', 'linestyle'], session=session)
            dash = 'solid' if style['linestyle'] == 'solid' else 'dash'
            fig.add_trace(go.Scatter(
                x=gps,
                y=points,
                mode='lines+markers',
                name=driver,
                line=dict(color=style['color'], dash=dash),
                marker=dict(symbol='circle')
            ))

        except KeyError:
            print(f"Style for driver {driver} not found")


    fig.update_layout(
        title=f'Championship Evolution of Drivers - {session.event.EventDate.year} Season',
        xaxis_title='Round',
        yaxis_title='Cumulative Points',
        legend_title='Driver',
        xaxis=dict(tickmode='array', tickvals=gps),
        yaxis=dict(gridcolor='lightgray', zerolinecolor='lightgray'),
        template='plotly_dark',
        width=900,
        height=600
    )

    fig.show()


In [332]:
plot_drivers_championship(df_cumulative_drivers_2022, session, top=10)

## Teams

In [334]:
df_cumulative_constructors_2022 = df_constructors_2022.fillna(0).cumsum(axis=1)

In [336]:
def plot_drivers_championship(df_cumulative, session, top=None):

    """wip"""

    gps = df_cumulative.columns
    if top:
        constructors = df_cumulative.index[:top]
    else:
        constructors = df_cumulative.index

    fig = go.Figure()

    for team in constructors:
        points = df_cumulative.loc[team, gps]
        
        try:
            color = fastf1.plotting.get_team_color(team, session=session)

            fig.add_trace(go.Scatter(
                x=gps,
                y=points,
                mode='lines+markers',
                name=team,
                line=dict(color=color),
                marker=dict(symbol='circle')
            ))

        except KeyError:
            print(f"Style for constructor {team} not found")

    fig.update_layout(
        title=f'Championship Evolution of Teams - {session.event.EventDate.year} Season',
        xaxis_title='Round',
        yaxis_title='Cumulative Points',
        legend_title='Team',
        xaxis=dict(tickmode='array', tickvals=gps),
        yaxis=dict(gridcolor='lightgray', zerolinecolor='lightgray'),
        template='plotly_dark',
        width=900,
        height=600
    )

    fig.show()
    

In [346]:
plot_drivers_championship(df_cumulative_constructors_2022, session, top=None)

---

## Grid

In [380]:
# Auxiliar dataframe with championship positions

def _get_df_championship(df_results):

    """wip, se le pasa el pivotado"""

    df_championship = pd.DataFrame({
        'championshipPosition': range(1, len(df_results) + 1),
        'driverCode': df_results.index
    })

    return df_championship

In [405]:
df_championship = _get_df_championship(df_drivers_2022)

In [216]:
# Posiciones de parrilla

ergast = Ergast()
races = ergast.get_race_schedule(2022)  # Races in year 2022
results_qualy = []

# For each race in the season
for rnd, race in races['raceName'].items():

    # Get results. Note that we use the round no. + 1, because the round no.
    # starts from one (1) instead of zero (0)
    temp = ergast.get_qualifying_results(season=2022, round=rnd + 1)
    temp = temp.content[0]

    # Add round no. and grand prix name
    temp['round'] = rnd + 1
    temp['race'] = race.removesuffix(' Grand Prix')
    temp = temp[['round', 'race', 'driverCode', 'position']]  # Keep useful cols.
    results_qualy.append(temp)

# Append all races into a single dataframe
results_qualy = pd.concat(results_qualy)
races = results['race'].drop_duplicates()

In [218]:
results_qualy_2 = results_qualy.pivot(index='driverCode', columns='round', values='position')

# Rank the drivers by theri championship position
results_qualy_2 = results_qualy_2.merge(df_championship, on='driverCode').set_index('driverCode').sort_values(by='championshipPosition').drop(columns='championshipPosition')

# Use race name, instead of round no., as column names
results_qualy_2.columns = races

### Delta

In [363]:
def get_results_delta(season):

    """wip"""

    ergast = Ergast()
    races = ergast.get_race_schedule(season)

    results = []

    # For each race in the season
    for rnd, race in races['raceName'].items():

        # Get results
        temp = ergast.get_race_results(season=season, round=rnd + 1).content[0]

        # Add round number and Grand Prix name
        temp['round'] = rnd + 1
        temp['race'] = race.removesuffix(' Grand Prix')
        temp = temp[['round', 'race', 'constructorName', 'driverCode', 'points', 'position', 'grid']]  # Keep useful cols.

        # Add to results
        results.append(temp)

    # Append all races into a single dataframe
    results = pd.concat(results)

    # Delta
    results['delta'] = results['grid'] - results['position']
    
    return results

In [364]:
df_delta = get_results_delta(2022)

In [365]:
df_delta

,round,race,constructorName,driverCode,points,position,grid,delta
0,1,Bahrain,Ferrari,LEC,26.0,1,1,0
1,1,Bahrain,Ferrari,SAI,18.0,2,3,1
2,1,Bahrain,Mercedes,HAM,15.0,3,5,2
3,1,Bahrain,Mercedes,RUS,12.0,4,9,5
4,1,Bahrain,Haas F1 Team,MAG,10.0,5,7,2
...,...,...,...,...,...,...,...,...
15,22,Abu Dhabi,Haas F1 Team,MSC,0.0,16,12,-4
16,22,Abu Dhabi,Haas F1 Team,MAG,0.0,17,16,-1
17,22,Abu Dhabi,Mercedes,HAM,0.0,18,5,-13
18,22,Abu Dhabi,Williams,LAT,0.0,19,20,1


In [412]:
df_championship = _get_df_championship(df_drivers_2022)

In [415]:
def get_chart(df_results, df_championship, values='delta'):

    """
    wip

    values can be 'delta', 'grid' or 'position'
    """

    df = df_results.pivot(index='driverCode', columns='round', values=values)
    df = df.merge(df_championship, on='driverCode').set_index('driverCode').sort_values(by='championshipPosition').drop(columns='championshipPosition')

    # Use race name, instead of round number, as column names
    df.columns = df_results['race'].drop_duplicates()

    return df


In [416]:
df = get_chart(df_delta, df_championship, values='delta')

In [419]:
df

race,Bahrain,Saudi Arabian,Australian,Emilia Romagna,Miami,Spanish,Monaco,Azerbaijan,Canadian,British,...,Hungarian,Belgian,Dutch,Italian,Singapore,Japanese,United States,Mexico City,São Paulo,Abu Dhabi
driverCode,,,,,,,,,,,,,,,,,,,,,
VER,-17.0,3.0,-16.0,0.0,2.0,1.0,1.0,2.0,0.0,-5.0,...,9.0,13.0,0.0,6.0,1.0,0.0,1.0,0.0,-3.0,0.0
LEC,0.0,0.0,0.0,-4.0,-1.0,-19.0,-3.0,-18.0,14.0,-1.0,...,-3.0,9.0,-1.0,-1.0,-1.0,-1.0,9.0,1.0,1.0,1.0
PER,-14.0,-3.0,1.0,1.0,0.0,3.0,2.0,0.0,-7.0,2.0,...,6.0,0.0,0.0,7.0,1.0,2.0,5.0,1.0,-3.0,-1.0
RUS,5.0,1.0,3.0,7.0,7.0,1.0,1.0,2.0,4.0,-10.0,...,-2.0,1.0,4.0,-1.0,-14.0,0.0,-1.0,-2.0,0.0,1.0
SAI,1.0,0.0,-11.0,-16.0,-1.0,-1.0,0.0,-16.0,1.0,0.0,...,-2.0,-2.0,-5.0,14.0,1.0,-16.0,-19.0,0.0,4.0,0.0
HAM,2.0,5.0,1.0,1.0,0.0,1.0,0.0,3.0,1.0,2.0,...,5.0,-16.0,0.0,14.0,-6.0,1.0,1.0,1.0,0.0,-13.0
NOR,-2.0,4.0,-1.0,2.0,-11.0,3.0,-1.0,2.0,-1.0,0.0,...,-3.0,5.0,0.0,-4.0,2.0,0.0,0.0,-1.0,-12.0,1.0
OCO,4.0,-1.0,1.0,2.0,12.0,5.0,-2.0,3.0,1.0,0.0,...,-4.0,9.0,3.0,3.0,1.0,1.0,-11.0,2.0,8.0,1.0
ALO,-1.0,-9.0,-7.0,-10.0,0.0,11.0,0.0,3.0,-7.0,2.0,...,-2.0,-2.0,7.0,-13.0,-13.0,0.0,7.0,-10.0,12.0,-10.0
